In [1]:
"""
Code for creation a table with different sheets where it writes sales managers responsible for country-sheet
with e-mail

"""

import pandas as pd
import numpy as np
import re

In [2]:
#raw initial data import

master = pd.read_excel('data_files/master_contacts.xlsx', sheet_name='master')
emails = pd.read_excel('data_files/master_contacts.xlsx', sheet_name='emails', header = None, names = ['Channel Manager','mail'])

# to search email

expression = '[A-z\._]+@[a-z]+\.[a-z]+.[a-z]+'

#new column with email

for n in range(len(emails)):
    emails.loc[n,'email'] = re.findall(expression, emails.iloc[n,1])[0]
    
emails = emails.drop(columns='mail')

# creation of table without duplicates with necessary columns, droping all empty lines
master = pd.merge(master,emails, how = 'left', on = 'Channel Manager')
master = master.loc[:,['Country', 'OPCO', 'Channel Manager', 'email']]
master.drop_duplicates(keep='first', inplace=True, ignore_index=True)
master.dropna(inplace = True)

# keeping unique countries only

countries_list = sorted(master.Country.unique())
countries_index = ['Index'] + countries_list
countries = pd.DataFrame(countries_index)

master['Role'] = 'Channel Manager'
master.rename(columns={'Channel Manager':'Name'},inplace = True)

master = master.loc[:,['OPCO', 'Name', 'Role', 'email', 'Country']]

# creation list of contacts 

list_of_contacts = []
list_of_contacts.append(countries)
for country in countries_list:
    mask = master.Country == country
    list_of_contacts.append(master[mask])

for n in range(len(list_of_contacts)):
    list_of_contacts[n].name = countries_index[n]

# final file creation 

writer = pd.ExcelWriter('data_files/Channel_Partner_Contacts.xlsx', engine='xlsxwriter')
for d in list_of_contacts:
    d.to_excel(writer, sheet_name = f'{d.name}')
    
writer.save()